Ссылка: https://www.youtube.com/watch?v=sIbzKA6MId8

# SQLAlchemy

`SQLAlchemy` - это библиотке `Python` для работы с БД, который использует `ORM`, который позволяет работать с базой при помощи языка `Python`, а не `SQL`.

## Определение модели

<b>Недекларативный способ</b> - сначала следует создание объекта таблицы, потом связываем ее с БД, а после связываем с классами, созданными нами.  
<b>Декларативный способ</b> - связка модели и таблицы происходит непосредственно в классе одновременно.  
Чаже всего используютДекларативный подход.

Вот пример создания БД:

In [1]:
import sqlalchemy as db 


metadata = db.MetaData() # информация о БД и ее объектах

authors = db.Table("authors", metadata,
    db.Column("id_author", db.Integer, primary_key=True),
    db.Column("name", db.String(250))
)

books = db.Table("books", metadata,
    db.Column("id_book", db.Integer, primary_key=True),
    db.Column("title", db.String(250), nullable=False),
    db.Column("author_id", db.Integer, db.ForeignKey("authors.id_author")),
    db.Column("genre", db.String(250)),
    db.Column("price", db.Integer)
)

engine = db.create_engine('sqlite:///books.db')
metadata.create_all(engine)

conneection = engine.connect()

insert_author_query = authors.insert().values([
    {"name": "Lutz"}
])
conneection.execute(insert_author_query)

insert_books_query = books.insert().values([
    {"title": "Learn Python", "author_id": 1, "genre": "Education", "price": 1299},
    {"title": "Clear Python", "author_id": 1, "genre": "Education", "price": 956},
])
conneection.execute(insert_books_query)



c:\temp1\ipykernel_12588\33293353.py:27: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conneection.execute(insert_author_query)


Получим данные:

In [2]:
books_gr_1000_query = books.select().where(books.columns.price > 1000)
result = conneection.execute(books_gr_1000_query)
print(result.fetchall())

[(1, 'Learn Python', 1, 'Education', 1299), (3, 'Learn Python', 1, 'Education', 1299), (5, 'Learn Python', 1, 'Education', 1299), (7, 'Learn Python', 1, 'Education', 1299)]


Еще получим данные:

In [3]:
join_select = db.select([authors, books]).where(authors.columns.id_author == books.columns.author_id)
result_join = conneection.execute(join_select)
print(result_join.fetchall())

[(1, 'Lutz', 1, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 2, 'Clear Python', 1, 'Education', 956), (1, 'Lutz', 3, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 4, 'Clear Python', 1, 'Education', 956), (1, 'Lutz', 5, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 6, 'Clear Python', 1, 'Education', 956), (1, 'Lutz', 7, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 8, 'Clear Python', 1, 'Education', 956)]


## Подключение и работа с существующей БД

Вот пример:

In [6]:
import sqlalchemy as db


engine = db.create_engine("sqlite:///books.db")
metadata = db.MetaData(engine) # помещаем данные БД в объект MetaData

# загружаем таблицы в переменные
authors = db.Table("authors", metadata, autoload=True)
books = db.Table("books", metadata, autoload=True)

# создаем подключение
connection = engine.connect()

# создание объекта выборки, обратите внимание на db.and_
select_query = db.select([authors, books]).where(db.and_(authors.columns.id_author == books.columns.author_id, books.columns.price > 1000))
result = connection.execute(select_query)
print(result.fetchall())

[(1, 'Lutz', 1, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 3, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 5, 'Learn Python', 1, 'Education', 1299), (1, 'Lutz', 7, 'Learn Python', 1, 'Education', 1299)]
